In [1]:
from qiskit import IBMQ
import numpy as np
from numpy import linalg
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute #, Aer
from qiskit import execute, IBMQ, BasicAer
from qiskit.providers.ibmq import least_busy
from tools import set_parameters
from launch import Launcher
from tools import BACKENDS

import cmath
import math
from scipy.linalg import expm
config = {
    'url': ''
}
IBMQ.enable_account('',**config)
# try:
#     IBMQ.load_accounts()
# except:
#     print("""WARNING: There's no connection with the API for remote backends.
#              Have you initialized a file with your personal token?
#              For now, there's only access to local simulator backends...""")

global BACKENDS
BACKENDS += IBMQ.backends()
print(IBMQ.backends())
parameters = {}
parameters['backend_name'] = 'ibmq_16_melbourne'
parameters['shots'] = 8192
launcher = Launcher(**{item: parameters[item] for item in ['backend_name', 'shots']})
#ROTATION
def ConditionedRotation(angle, controllingQbit, subjectedQbit, circuit, quantReg):
  
    circuit.u3(angle, 0, 0, quantReg[subjectedQbit])
    circuit.z(quantReg[subjectedQbit])
    circuit.cx(quantReg[controllingQbit], quantReg[subjectedQbit])
    circuit.u3( ( - angle ), 0, 0, quantReg[subjectedQbit])
    circuit.z(quantReg[subjectedQbit])

t = 0.25    
n = 2
x = np.array([[2, 1], [1, 2]])

def get_abgd_from_unitary_matrix(U):
    if abs(U[0][0]) < 1e-4:
        gamma = np.pi
        delta = 0
        alpha = 0.5*cmath.phase(-U[1,0]*U[0,1])
        beta = 2*(cmath.phase(U[1,0]) - alpha)
    elif abs(U[1][0]) < 1e-4:
        gamma = 0
        delta = 0
        alpha = 0.5*cmath.phase(U[0,0]*U[1,1])
        beta = 2*(cmath.phase(U[1,1]) - alpha)
    else:
        alpha = 0.5*cmath.phase(U[0][0]*U[1][1])
        beta = cmath.phase(U[1][0]/U[0][0])
        gamma = 2*math.asin(abs(U[1][0]))
        delta = (cmath.phase(U[1][1]/U[0][0]) - beta)
    return alpha, beta, gamma, delta

y = expm(2*1j*np.pi*t*x)
a, b, c, d = get_abgd_from_unitary_matrix(y)

y_2 = np.array(y)@np.array(y)
a1, b1, c1, d1 = get_abgd_from_unitary_matrix(y_2)

    
solution = np.array([math.sqrt(0.5), math.sqrt(0.5)])
# solution = np.array([2, -1])/np.sqrt(5)
solution_T = np.conj(solution.T)


# Define the Quantum and Classical Registers
qr = QuantumRegister(14)
#cr = ClassicalRegister(14)
# Build the circuits
circuit = QuantumCircuit(qr)
#b
circuit.h(qr[8])

#U
circuit.h(qr[9])
circuit.u1((d - b)/2, qr[8]) #C
circuit.cx(qr[9], qr[8]) #CNOT
circuit.u1(-(d+b)/2, qr[8])
circuit.u3(-c/2, 0, 0, qr[8])
circuit.cx(qr[9], qr[8]) #CNOT
circuit.u3(c/2, 0, 0, qr[8])
circuit.u1(b, qr[8])
circuit.u1(a, qr[9]) #a

# U^2
circuit.h(qr[5])
circuit.u1((d1 - b1)/2, qr[8]) #C
circuit.swap(qr[5], qr[6]) 
circuit.cx(qr[6], qr[8])
circuit.swap(qr[5], qr[6])
circuit.u1(-(d1+b1)/2, qr[8])
circuit.u3(-c1/2, 0, 0, qr[8])
circuit.swap(qr[5], qr[6]) 
circuit.cx(qr[6], qr[8])
circuit.swap(qr[5], qr[6])
circuit.u3(c1/2, 0, 0, qr[8])
circuit.u1(b1, qr[8])
circuit.u1(a1, qr[5]) #a

# FT.inverse()
circuit.h(qr[5])
circuit.u1(-(np.pi/4), qr[5])
circuit.u1(-np.pi/2, qr[9])
circuit.cx(qr[5], qr[9])
circuit.u1((np.pi/4), qr[9])
circuit.cx(qr[5], qr[9])
circuit.u1((np.pi/4), qr[9])
circuit.h(qr[9])


# from qiskit import execute

# circuit.measure(qr[2], cr[2])
# circuit.measure(qr[3], cr[3])
# print(execute(circuits=[circuit], backend=BasicAer.get_backend('qasm_simulator'), shots=8192).result().get_counts())
# import sys; sys.exit(0)


#R
circuit.u3(np.pi/4, 0, 0, qr[10])
circuit.z(qr[10])
circuit.swap(qr[5], qr[4]) 
circuit.cx(qr[4], qr[10])
circuit.swap(qr[5], qr[4])
circuit.u3( ( - np.pi/4 ), 0, 0, qr[10])
circuit.z(qr[10])
ConditionedRotation(np.pi/8, 9, 10, circuit, qr)

#FT
circuit.h(qr[9])
circuit.u1(-(np.pi/4), qr[9])
circuit.cx(qr[5], qr[9])
circuit.u1(-(np.pi/4), qr[9])
circuit.cx(qr[5], qr[9])
circuit.u1(np.pi/2, qr[9])
circuit.u1((np.pi/4), qr[2])
circuit.h(qr[2])

circuit.u1(-a1, qr[5])
circuit.u1(-b1, qr[8])
circuit.u3(-c1/2, 0, 0, qr[8])
circuit.swap(qr[5], qr[6]) 
circuit.cx(qr[6], qr[8])
circuit.swap(qr[5], qr[6])
circuit.u3(c1/2, 0, 0, qr[8])
circuit.u1((d1+b1)/2, qr[8])
circuit.swap(qr[5], qr[6]) 
circuit.cx(qr[6], qr[8])
circuit.swap(qr[5], qr[6])
circuit.u1(-(d1 - b1)/2, qr[8]) #C
circuit.h(qr[5])


circuit.u1(-a, qr[9])
circuit.u1(-b, qr[8])
circuit.u3(-c/2, 0, 0, qr[8])
circuit.cx(qr[9], qr[8])
circuit.u3(c/2, 0, 0, qr[8])
circuit.u1((d+b)/2, qr[8])
circuit.cx(qr[9], qr[8])
circuit.u1(-(d - b)/2, qr[8]) #C
circuit.h(qr[9])


# meas_qubits - кубиты, использующиеся для томографии. Порядок важен (для [0, 1] и [1, 0] ответ будет разным)
exp_solution_4 = launcher.run([circuit], meas_qubits=[10, 8])[0]
# print(exp_solution_4)

exp_solution_4 = np.kron(np.eye(2), np.diag([0, 1]))@exp_solution_4@np.kron(np.eye(2), np.diag([0, 1]))

exp_solution_4 = exp_solution_4/np.trace(exp_solution_4)

exp_solution_2 = np.array([
    [exp_solution_4[0,0] + exp_solution_4[1,1], exp_solution_4[0,2] + exp_solution_4[1,3]],
    [exp_solution_4[2,0] + exp_solution_4[3,1],exp_solution_4[2,2] + exp_solution_4[3,3]]
])

# print(exp_solution_2)
prob = solution @ exp_solution_2 @ solution_T
print(prob) 


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(ibm-q, open, main)>, <IBMQBackend('ibmqx4') from IBMQ(ibm-q, open, main)>, <IBMQBackend('ibmqx2') from IBMQ(ibm-q, open, main)>, <IBMQBackend('ibmq_16_melbourne') from IBMQ(ibm-q, open, main)>]
(0.48326651955601896+0j)


In [3]:
circuit.draw()